In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [7]:
# from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama


llm = ChatOllama(model="mistral", temperature=0)

In [ ]:
llm.invoke("How will the weather be in munich today?")

In [9]:
from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "munich" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "munich" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

In [10]:
llm_with_tools = llm.bind_tools(tools)

In [12]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-02-27T08:20:19.3091893Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3570875900, 'load_duration': 10680600, 'prompt_eval_count': 151, 'prompt_eval_duration': 276000000, 'eval_count': 109, 'eval_duration': 3281000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-9363950b-6b93-49f7-8e7b-c96e14730810-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'munich'}, 'id': '6bd80c4f-551a-444d-9012-8674b932b6d4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 109, 'total_tokens': 260})

In [13]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'munich'},
  'id': '6bd80c4f-551a-444d-9012-8674b932b6d4',
  'type': 'tool_call'}]

In [ ]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

In [ ]:
result.tool_calls

In [14]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [ ]:
messages

In [15]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [16]:
llm_output.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'ad30590e-bb62-45e8-91f8-80ea2c0ac3e6',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Munich', 'seating_type': 'outdoor'},
  'id': '49b692b1-1ca7-4ff8-bfd6-f16939e44b7e',
  'type': 'tool_call'},
 {'name': 'get_weather',
  'args': {'location': 'Munich'},
  'id': 'dc8023fb-ff0b-40f9-a1fb-93d91d9f11b2',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Munich', 'seating_type': 'outdoor'},
  'id': '6fe5a4cd-1f04-4818-8405-3b3d4f42f209',
  'type': 'tool_call'}]

In [17]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

There are actually two ways to handle function calling with bind_tools in LangChain:

🔹 1. LLM Generates Tool Calls, But You Manually Execute Them (Your Code)
In your current implementation, the LLM only decides which tool to call and provides the parameters, but you still need to:

Extract the tool call from llm_output.tool_calls
Manually invoke the tool function
Pass the tool’s result back to the LLM
This means LangChain is not automatically executing the tool calls—you are.

🔹 Why is this happening?

The LLM only returns structured tool call requests in tool_calls (like a JSON output).
But LangChain does not execute them for you unless you explicitly integrate an execution step.
f

🔹 2. Let LangChain Execute the Tools Automatically
To make LangChain automatically execute the tools, use:

llm_with_tools = llm.bind_tools(tools, auto_execute=True)

In [ ]:
messages

In [18]:
llm_with_tools.invoke(messages)

AIMessage(content=" It's currently 15 degrees Celsius and cloudy in Munich. We still have outdoor seating available. Enjoy your day!", additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-02-27T08:21:04.6219109Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1358507700, 'load_duration': 10425600, 'prompt_eval_count': 192, 'prompt_eval_duration': 307000000, 'eval_count': 31, 'eval_duration': 1006000000, 'message': Message(role='assistant', content=" It's currently 15 degrees Celsius and cloudy in Munich. We still have outdoor seating available. Enjoy your day!", images=None, tool_calls=None)}, id='run-0a65d694-5ff9-4a8b-b073-4bc8cebb6417-0', usage_metadata={'input_tokens': 192, 'output_tokens': 31, 'total_tokens': 223})